In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ============================================================
# 0) 최소 패키지 설치  (TF 2.18.x는 Colab 기본 그대로)
# ============================================================
!pip install --quiet tqdm haversine

import numpy as np, pandas as pd, tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from haversine import haversine
from tqdm import tqdm
from pathlib import Path

# ------------------ 하이퍼파라미터 --------------------------
SEQ_LEN   = 60
FEATS     = ['dlat','dlon','sog','cog','heading','dt']
D_MODEL   = 128; N_HEAD = 4; N_LAYER = 4
MASK_RATE = 0.5
BATCH     = 256; EPOCHS = 40
TH_PCTL   = 95
CSV_PATH  = "/content/drive/MyDrive/25년 해군 AI 경진대회/AIS_2024_01_05.csv"
# ------------------------------------------------------------

# ============================================================
# 1) 데이터 로딩 & 기초 전처리
# ============================================================
cols = ['MMSI','BaseDateTime','LAT','LON','SOG','COG','Heading']
df   = (pd.read_csv(CSV_PATH, usecols=cols, parse_dates=['BaseDateTime'])
          .drop_duplicates(['MMSI','BaseDateTime'])
          .sort_values(['MMSI','BaseDateTime'])
          .reset_index(drop=True))

df['SOG']     = df['SOG'].replace(102.3, np.nan)
df['Heading'] = df['Heading'].replace(511,   np.nan)

num_cols = ['LAT','LON','SOG','COG','Heading']
df[num_cols] = (df.groupby('MMSI')[num_cols]
                  .transform(lambda s: s.interpolate().ffill().bfill()))

df = df[df['SOG'].between(0,80)]

# ============================================================
# 2) 파생 변수
# ============================================================
for new, src in [('dlat','LAT'), ('dlon','LON')]:
    df[new] = df.groupby('MMSI')[src].transform(lambda s: s.diff().fillna(0))

df['dt']      = (df.groupby('MMSI')['BaseDateTime']
                   .transform(lambda s: s.diff().dt.total_seconds().fillna(0)))
df['heading'] = df['Heading'].fillna(0)
df['cog']     = df['COG'].fillna(0)
df['sog']     = df['SOG'].fillna(0)

# ============================================================
# 3) 북한 선박 제외 & Train/Val/Test 분할
# ============================================================
is_dprk   = df['MMSI'].astype(str).str.startswith(('445','447'))
normal_df = df[~is_dprk]

ships = normal_df['MMSI'].unique()
tr_ids, tmp = train_test_split(ships, test_size=0.30, random_state=42)
va_ids, te_ids = train_test_split(tmp, test_size=0.50, random_state=42)

def pick(ids): return normal_df[normal_df['MMSI'].isin(ids)]
tr_df, va_df, te_df = map(pick, (tr_ids, va_ids, te_ids))

# ============================================================
# 4) 윈도우 & 스케일링
# ============================================================
def make_windows(df, seq=SEQ_LEN):
    X=[]
    for _,g in df.groupby('MMSI'):
        A=g[FEATS].values
        for i in range(len(A)-seq+1):
            X.append(A[i:i+seq])
    return np.stack(X, dtype=np.float32)

tr_X, va_X, te_X = map(make_windows, (tr_df, va_df, te_df))

scaler = StandardScaler().fit(tr_X.reshape(-1,len(FEATS)))
def scale(x): return scaler.transform(x.reshape(-1,len(FEATS))).reshape(x.shape)
tr_X, va_X, te_X = map(scale, (tr_X, va_X, te_X))

print("Shapes:", tr_X.shape, va_X.shape, te_X.shape)

# ============================================================
# 5) Positional Encoding & RandomMask 레이어
# ============================================================
class SinePos(tf.keras.layers.Layer):
    def __init__(self, length, dim, **kw):
        super().__init__(**kw)
        pos = np.arange(length)[:,None]
        i   = np.arange(dim)[None,:]
        angle = pos / np.power(10000, (2*(i//2))/dim)
        pe = np.zeros((length,dim))
        pe[:,0::2]=np.sin(angle[:,0::2])
        pe[:,1::2]=np.cos(angle[:,1::2])
        self.pe = tf.constant(pe, dtype=tf.float32)[None,:,:]
    def call(self,x):
        return x + self.pe

class RandomMask(tf.keras.layers.Layer):
    def __init__(self, rate, **kw):
        super().__init__(**kw); self.rate=rate
    def call(self, x, training=None):
        if training:
            mask=tf.cast(tf.random.uniform(tf.shape(x)[:2])<self.rate, x.dtype)
            mask=tf.expand_dims(mask,-1)
            return x*(1-mask)    # masked tokens → 0
        return x

# ============================================================
# 6) Transformer-based Masked AutoEncoder
# ============================================================
def build_mae(seq_len, n_feat, d_model, n_head, n_layer, mask_rate):
    inp = tf.keras.Input(shape=(seq_len, n_feat))
    x   = tf.keras.layers.Dense(d_model)(inp)
    x   = SinePos(seq_len, d_model)(x)
    x   = RandomMask(mask_rate)(x)          # <-- Keras Layer이므로 오류 없음

    for _ in range(n_layer):
        attn = tf.keras.layers.MultiHeadAttention(
            num_heads=n_head, key_dim=d_model//n_head)(x,x)
        x = tf.keras.layers.LayerNormalization()(x+attn)
        ff = tf.keras.Sequential([
            tf.keras.layers.Dense(4*d_model, activation='gelu'),
            tf.keras.layers.Dense(d_model)
        ])(x)
        x = tf.keras.layers.LayerNormalization()(x+ff)

    out = tf.keras.layers.Dense(n_feat)(x)
    return tf.keras.Model(inp, out, name="MaskedAE")

model = build_mae(SEQ_LEN,len(FEATS),D_MODEL,N_HEAD,N_LAYER,MASK_RATE)
model.compile(optimizer='adam', loss='mse')
model.summary()

# ============================================================
# 7) 학습
# ============================================================
_ = model.fit(tr_X, tr_X,
              validation_data=(va_X, va_X),
              epochs=EPOCHS, batch_size=BATCH,
              callbacks=[tf.keras.callbacks.EarlyStopping(
                  patience=5, restore_best_weights=True)])

# ============================================================
# 8) 임계값 & 테스트 평가
# ============================================================
mse_tr = np.mean((tr_X - model.predict(tr_X,BATCH))**2, axis=(1,2))
τ      = np.percentile(mse_tr, TH_PCTL)
print(f"τ({TH_PCTL}%) = {τ:.6f}")

mse_te = np.mean((te_X - model.predict(te_X,BATCH))**2, axis=(1,2))
y_pred = (mse_te > τ).astype(int)
print(f"Test windows: {len(te_X)},  anomaly ratio: {y_pred.mean()*100:.2f}%")

# ============================================================
# 9) 저장
# ============================================================
Path("/content/mae_model").mkdir(exist_ok=True)
model.save("/content/mae_model/mae.h5")
np.save("/content/mae_model/scaler_mean.npy", scaler.mean_)
np.save("/content/mae_model/scaler_scale.npy", scaler.scale_)
print("✅ saved to /content/mae_model")

Shapes: (4630188, 60, 6) (979166, 60, 6) (992032, 60, 6)


Model: "MaskedAE"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 60, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 60, 128)   │        896 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sine_pos (SinePos)  │ (None, 60, 128)   │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_mask         │ (None, 60, 128)   │          0 │ sine_pos[0][0]    │
│ (RandomMask)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 128)   │     66,048 │ random_mask[0][0… │
│ (MultiHeadAttentio… │                   │            │ random_mask[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 60, 128)   │          0 │ random_mask[0][0… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 60, 128)   │        256 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 60, 128)   │    131,712 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 60, 128)   │          0 │ layer_normalizat… │
│                     │                   │            │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 128)   │        256 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 128)   │     66,048 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 60, 128)   │          0 │ layer_normalizat… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 128)   │        256 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 60, 128)   │    131,712 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 60, 128)   │          0 │ layer_normalizat… │
│                     │                   │            │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 128)   │        256 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 128)   │     66,048 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 794,758 (3.03 MB)

 Trainable params: 794,758 (3.03 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 215s 11ms/step - loss: 0.4856 - val_loss: 1.0108
Epoch 2/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 173s 10ms/step - loss: 0.5371 - val_loss: 1.2377
Epoch 3/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 173s 10ms/step - loss: 0.5448 - val_loss: 0.8239
Epoch 4/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 173s 10ms/step - loss: 0.5097 - val_loss: 0.9478
Epoch 5/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 177s 10ms/step - loss: 0.5185 - val_loss: 14.3429
Epoch 6/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 177s 10ms/step - loss: 0.4536 - val_loss: 0.9988
Epoch 7/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 174s 10ms/step - loss: 0.5194 - val_loss: 1.0999
Epoch 8/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 173s 10ms/step - loss: 0.5425 - val_loss: 0.7017
Epoch 9/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 174s 10ms/step - loss: 0.6008 - val_loss: 0.5940
Epoch 10/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 177s 10ms/step - loss: 0.5936 - val_loss: 0.8680
Epoch 11/40
18087/18087 ━━━━━━━━━━━━━━━━━━━━ 174s 10ms/step - loss: 

Test windows: 992032,  anomaly ratio: 5.11%
✅ saved to /content/mae_model
